In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.conf import SparkConf
config = SparkConf()
config.setMaster("local").setAppName("DataFrameAsg01")

from pyspark.sql import SparkSession
spark = SparkSession.builder.config(conf=config).getOrCreate()

sc= spark.sparkContext

22/03/12 05:22:27 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/12 05:22:27 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
22/03/12 05:22:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
#SYMBOL,SERIES,OPEN,HIGH,LOW,CLOSE,LAST,PREVCLOSE,TOTTRDQTY,TOTTRDVAL,TIMESTAMP,TOTALTRADES,ISIN,
from pyspark.sql.types import StructType, LongType, StringType, IntegerType, DoubleType, DateType, TimestampType

stockSchema = StructType()\
                    .add("SYMBOL", StringType(),True)\
                    .add("SERIES", StringType(),True)\
                    .add("OPEN", DoubleType(),True)\
                    .add("HIGH", DoubleType(),True)\
                    .add("LOW", DoubleType(),True)\
                    .add("CLOSE", DoubleType(),True)\
                    .add("LAST", DoubleType(),True)\
                    .add("PREVCLOSE", DoubleType(),True)\
                    .add("TOTTRDQTY", IntegerType(),True)\
                    .add("TOTTRDVAL", DoubleType(),True)\
                    .add("TIMESTAMP", StringType(),True)\
                    .add("TOTALTRADES", IntegerType(),True)\
                    .add("ISIN", StringType(),True)


In [4]:
stockDf= spark.read.format("csv").option("header", True).schema(stockSchema).load("hdfs://localhost:9000/asgStocks")
stockDf.printSchema()

stockDf.show(3)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)



+----------+------+----+-----+-----+-----+-----+---------+---------+-------------+-----------+-----------+------------+
|    SYMBOL|SERIES|OPEN| HIGH|  LOW|CLOSE| LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|
+----------+------+----+-----+-----+-----+-----+---------+---------+-------------+-----------+-----------+------------+
| 20MICRONS|    EQ|76.0|77.55|73.25| 74.1| 73.6|     77.2|   141872|1.061998695E7|21-FEB-2022|       2443|INE144J01027|
|21STCENMGM|    EQ|33.3| 33.3| 33.3| 33.3| 33.3|    33.95|      415|      13819.5|21-FEB-2022|         24|INE253B01015|
| 3IINFOLTD|    EQ|62.0| 63.4|59.05|59.05|59.05|    62.15|   753815| 4.47928636E7|21-FEB-2022|       4999|INE748C01038|
+----------+------+----+-----+-----+-----+-----+---------+---------+-------------+-----------+-----------+------------+
only showing top 3 rows



22/03/12 05:22:42 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 13
CSV file: hdfs://localhost:9000/asgStocks/cm21FEB2022bhav.csv


In [5]:
from pyspark.sql.functions import *
stockDateDf= stockDf.withColumn("Date" , to_timestamp(col("TIMESTAMP"),"dd-MMM-yyyy"))
stockDateDf.printSchema()

stockDateDf.show(3)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)
 |-- Date: timestamp (nullable = true)

+----------+------+----+-----+-----+-----+-----+---------+---------+-------------+-----------+-----------+------------+-------------------+
|    SYMBOL|SERIES|OPEN| HIGH|  LOW|CLOSE| LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|               Date|
+----------+------+----+-----+-----+-----+-----+---------+---------+-------------+-----------+-----------+------------+-------------------+
| 20MICRONS|    EQ|76.0|77.55|73.2

22/03/12 05:22:47 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 13
CSV file: hdfs://localhost:9000/asgStocks/cm21FEB2022bhav.csv


In [6]:
stockFinalDf = stockDateDf.withColumn("YEAR", date_format(col("Date"), "yyyy"))\
                            .withColumn("MONTH", date_format(col("Date"), "MM"))\
                            .withColumn("DAY", date_format(col("Date"), "dd"))
stockFinalDf.printSchema()
stockFinalDf.show(5)

root
 |-- SYMBOL: string (nullable = true)
 |-- SERIES: string (nullable = true)
 |-- OPEN: double (nullable = true)
 |-- HIGH: double (nullable = true)
 |-- LOW: double (nullable = true)
 |-- CLOSE: double (nullable = true)
 |-- LAST: double (nullable = true)
 |-- PREVCLOSE: double (nullable = true)
 |-- TOTTRDQTY: integer (nullable = true)
 |-- TOTTRDVAL: double (nullable = true)
 |-- TIMESTAMP: string (nullable = true)
 |-- TOTALTRADES: integer (nullable = true)
 |-- ISIN: string (nullable = true)
 |-- Date: timestamp (nullable = true)
 |-- YEAR: string (nullable = true)
 |-- MONTH: string (nullable = true)
 |-- DAY: string (nullable = true)

+----------+------+-------+--------+--------+-------+-------+---------+---------+-------------+-----------+-----------+------------+-------------------+----+-----+---+
|    SYMBOL|SERIES|   OPEN|    HIGH|     LOW|  CLOSE|   LAST|PREVCLOSE|TOTTRDQTY|    TOTTRDVAL|  TIMESTAMP|TOTALTRADES|        ISIN|               Date|YEAR|MONTH|DAY|
+---------

22/03/12 05:22:55 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 13
CSV file: hdfs://localhost:9000/asgStocks/cm21FEB2022bhav.csv


In [7]:
stockFinalDf.write\
            .partitionBy("YEAR", "MONTH", "DAY")\
            .format("parquet")\
            .mode("overwrite")\
            .save("hdfs://localhost:9000/Assignment01StockWrite/")

22/03/12 05:23:03 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 13
CSV file: hdfs://localhost:9000/asgStocks/cm21FEB2022bhav.csv
22/03/12 05:23:03 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 13
CSV file: hdfs://localhost:9000/asgStocks/cm24FEB2022bhav.csv
22/03/12 05:23:03 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 13
CSV file: hdfs://localhost:9000/asgStocks/cm02MAR2022bhav.csv
22/03/12 05:23:03 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header length: 14, schema size: 13
CSV file: hdfs://localhost:9000/asgStocks/cm28FEB2022bhav.csv
22/03/12 05:23:04 WARN CSVHeaderChecker: Number of column in CSV header is not equal to number of fields in the schema:
 Header 